In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough

### Model

In [5]:
model = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    api_key=open('api.txt', 'r').read()
)

### Prompts

In [15]:
# template = """You are a helpful assistant who generates comma separated lists.
# A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
# ONLY return a comma separated list, and nothing more."""
# human_template = "{text}"

# chat_prompt = ChatPromptTemplate.from_messages([
#     ("system", template),
#     ("human", human_template),
# ])

In [7]:
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

### Parser

In [8]:
model_parser = model | StrOutputParser()

### Chain

In [9]:
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser

question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

### Run

In [10]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of pomegranate and the flag of Armenia?')])

In [11]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color coral is a reddish or pinkish shade of orange. \n\nThe flag of Comoros consists of four horizontal stripes of yellow, white, red, and blue, from top to bottom.')

If Branching is intersting: https://python.langchain.com/docs/expression_language/cookbook/multiple_chains